In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline, GenerationConfig

In [2]:
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/msmarco-distilbert-base-tas-b")

# Load the FAISS store from disk
faiss_store = FAISS.load_local("../data/faiss_db_4000", hf_embeddings, allow_dangerous_deserialization=True)

/var/folders/yv/gpc5sh7j5k3cls_fv_dpzntc0000gn/T/ipykernel_8261/878308936.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/msmarco-distilbert-base-tas-b")


In [6]:
model_id = "google/flan-t5-large"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

gen_cfg = GenerationConfig.from_pretrained(model_id)
gen_cfg.max_new_tokens=512
gen_cfg.temperature=0.0000001 # 0.0
gen_cfg.return_full_text=True
gen_cfg.do_sample=True
gen_cfg.repetition_penalty=1.11

# Build a text2text-generation pipeline
pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=gen_cfg
)

# Wrap the pipeline with LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use mps:0


In [11]:
prompt_template_str = """You are a timeline-based Question Answering system. Answer the question below based on the provided context.

Context:
{context}

Question: {question}

Answer:
"""

prompt = PromptTemplate(
    template=prompt_template_str, 
    input_variables=["context", "question"]
)

In [12]:
def answer_tlqa(question: str, k: int = 3) -> str:
    """
    Retrieves the top-k most relevant chunks from the FAISS store,
    formats them into a prompt for FlanT5, 
    and returns the generative answer.
    """
    # Retrieve top-k documents
    docs = faiss_store.similarity_search(question, k=k)

    # Combine the retrieved texts into one context string
    context = "\n".join([doc.page_content for doc in docs])
    print(context)

    # Format the prompt
    final_prompt = prompt.format(context=context, question=question)

    # Generate an answer using the LLM
    response = llm(final_prompt)

    # The pipeline returns a list of dictionaries or a string. 
    # If using LangChain wrapper, it might return a string directly.
    # Adjust as necessary if your pipeline returns a different format.
    return response

In [15]:
# Replace this with any TLQA-style question you have
test_question = "Who is Diego Borja?"
k_value = 1

answer = answer_tlqa(test_question, k=k_value)
print("Question:", test_question)
print("Top-k Context Used:", k_value)
print("Answer:", answer)

== Political Leader ==
Diego Borja is the Coordination Minister for Economic Policy of Ecuador, appointed on December 23, 2008. He is also National President of the "Movimiento Poder Ciudadano" (Citizen's Power Movement). He was elected a member of the Constitutional Assembly on September 30, 2007, as a candidate of the alliance between the social-democratic "Izquierda Democrática" (Party of the Democratic Left), the Poder Ciudadano Movement, and two other democratic socialist movements, "Acuerdo Democrático" and "Nuevo Acuerdo Nacional". He was elected as the first candidate in the Province of Pichincha, which includes the capital city of Quito.
Before forming "Movimiento Poder Ciudadano", Borja was a national leader of "Movimiento Nuevo País" (New Country Movement), where he was an adviser to Presidential candidate Freddy Ehlers, as well as alternate representative from Ecuador to the Andean Parliament.
Question: Who is Diego Borja?
Top-k Context Used: 1
Answer: (2008, 2009)
